In [5]:
# Fixing syntax error and re-running generation.
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import math, random, csv, json, os
from collections import defaultdict

random.seed(12345)

GPX_PATH = "Gunung Prau.gpx"
OUT_DIR = "data"
DEVICE_TRACK_CSV = os.path.join(OUT_DIR, "device_track.csv")
ALERTS_CSV = os.path.join(OUT_DIR, "alert_events.csv")
EMERGENCY_CSV = os.path.join(OUT_DIR, "emergency_events.csv")
TRAILS_CSV = os.path.join(OUT_DIR, "trails.csv")

DATE_STR = "2025-11-11"
DAY_START = datetime.fromisoformat(DATE_STR + "T00:00:00")
DAY_END = datetime.fromisoformat(DATE_STR + "T23:59:00")

weather_timeline = [
    ("00:00", "01:00", "light_rain"),
    ("01:00", "07:00", "cloudy"),
    ("07:00", "15:00", "thunderstorm"),
    ("15:00", "17:00", "light_rain"),
    ("17:00", "21:00", "clear"),
    ("21:00", "24:00", "cloudy"),
]

def parse_weather(ts):
    t = ts.time()
    for start_s, end_s, cond in weather_timeline:
        s_h, s_m = map(int, start_s.split(":"))
        e_h, e_m = map(int, end_s.split(":"))
        start = datetime(ts.year, ts.month, ts.day, s_h, s_m).time()
        end = datetime(ts.year, ts.month, ts.day, e_h % 24, e_m).time()
        if start <= t < end or (start > end and (t >= start or t < end)):
            return cond
    return "unknown"

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2-lat1); dlambda = math.radians(lon2-lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R*c

def move_point(lat, lon, bearing_deg, distance_m):
    R = 6378137.0
    bearing = math.radians(bearing_deg)
    lat1 = math.radians(lat); lon1 = math.radians(lon)
    lat2 = math.asin(math.sin(lat1)*math.cos(distance_m/R) + math.cos(lat1)*math.sin(distance_m/R)*math.cos(bearing))
    lon2 = lon1 + math.atan2(math.sin(bearing)*math.sin(distance_m/R)*math.cos(lat1), math.cos(distance_m/R)-math.sin(lat1)*math.sin(lat2))
    return math.degrees(lat2), math.degrees(lon2)

def lateral_shift(lat, lon, lat_next, lon_next, offset_m):
    dlon = math.radians(lon_next - lon)
    lat1 = math.radians(lat); lat2 = math.radians(lat_next)
    y = math.sin(dlon) * math.cos(lat2)
    x = math.cos(lat1)*math.sin(lat2) - math.sin(lat1)*math.cos(lat2)*math.cos(dlon)
    bearing = (math.degrees(math.atan2(y, x)) + 360) % 360
    perp = (bearing + 90) % 360 if random.random() < 0.5 else (bearing - 90) % 360
    return move_point(lat, lon, perp, offset_m)

# parse GPX
tree = ET.parse(GPX_PATH)
root = tree.getroot()
ns = {"g": "http://www.topografix.com/GPX/1/1"}
tracks = []
for trk in root.findall("g:trk", ns):
    name_el = trk.find("g:name", ns); trk_name = name_el.text.strip() if name_el is not None else "unnamed"
    coords = []
    for trkseg in trk.findall("g:trkseg", ns):
        for pt in trkseg.findall("g:trkpt", ns):
            lat = float(pt.attrib['lat']); lon = float(pt.attrib['lon'])
            coords.append((lat, lon))
    tracks.append({"name": trk_name.lower(), "coords": coords})

waypoints = []
for wpt in root.findall("g:wpt", ns):
    name_el = wpt.find("g:name", ns); name = name_el.text.strip().lower() if name_el is not None else ""
    lat = float(wpt.attrib['lat']); lon = float(wpt.attrib['lon'])
    waypoints.append({"name": name, "lat": lat, "lon": lon})

# Map trails
required = ["wates", "patakbanteng", "dieng"]
trail_map = {}
used = set()
for req in required:
    found = None
    for i, t in enumerate(tracks):
        if req in t["name"] and i not in used:
            found = i; break
    if found is None:
        for i, t in enumerate(tracks):
            if t["name"].startswith(req[:4]) and i not in used:
                found = i; break
    if found is None:
        for i in range(len(tracks)):
            if i not in used:
                found = i; break
    trail_map[req] = tracks[found]; used.add(found)

# build segs & cumdist
def build_trail_meta(coords):
    segs = []; cum = [0.0]; total = 0.0
    for i in range(len(coords)-1):
        lat1, lon1 = coords[i]; lat2, lon2 = coords[i+1]
        d = haversine(lat1, lon1, lat2, lon2)
        segs.append({"start": coords[i], "end": coords[i+1], "length": d})
        total += d; cum.append(total)
    return segs, cum, total

trail_data = {}
for k,t in trail_map.items():
    segs, cumdist, total = build_trail_meta(t["coords"])
    trail_data[k] = {"name": t["name"], "coords": t["coords"], "segs": segs, "cumdist": cumdist, "length_m": total}

# nearest index helper
def find_nearest_index(trail_key, lat, lon):
    coords = trail_data[trail_key]["coords"]
    best_i = 0; best_d = float("inf")
    for i, (plat, plon) in enumerate(coords):
        d = haversine(lat, lon, plat, plon)
        if d < best_d:
            best_d = d; best_i = i
    dist_along = trail_data[trail_key]["cumdist"][best_i] if best_i < len(trail_data[trail_key]["cumdist"]) else 0.0
    return best_i, dist_along

# basecamp finder
def find_basecamp_for(trail_key):
    for wp in waypoints:
        if "basecamp" in wp["name"] and trail_key in wp["name"]:
            return wp
    for wp in waypoints:
        if "basecamp" in wp["name"]:
            return wp
    return None

basepoints = {}
for k in required:
    basepoints[k] = find_basecamp_for(k)

device_counts = {"wates": 223, "dieng": 307, "patakbanteng": 398}
config = {
    "wates": {"off_pct": 0.05, "stuck_pct_of_off": 0.05, "sos_count": 2, "idle_n": 53, "idle_min": 10},
    "dieng": {"off_pct": 0.03, "stuck_pct_of_off": 0.01, "sos_count": 0, "idle_n": 48, "idle_min": 5},
    "patakbanteng": {"off_pct": 0.04, "stuck_pct_of_off": 0.02, "sos_count": 1, "idle_n": 36, "idle_min": 7},
}

weather_speed_mul = {"clear":1.0,"cloudy":0.95,"light_rain":0.85,"thunderstorm":0.6}
def sample_start_time():
    r = random.random()
    if r < 0.60:
        minute = random.randint(60, 7*60 - 1)
    elif r < 0.70:
        minute = random.randint(7*60, 15*60 - 1)
    else:
        if random.random() < 0.5:
            minute = random.randint(0, 59)
        else:
            minute = random.randint(15*60, 24*60 - 1)
    return DAY_START + timedelta(minutes=minute, seconds=random.randint(0,59))

devices = {}
device_list_by_trail = defaultdict(list)
def create_devices_for_trail(trail_key, count):
    devices_local = []
    num_grouped = int(count * 0.8)
    num_solo = count - num_grouped
    grouped_created = 0; group_id = 0
    base_wp = basepoints.get(trail_key)
    if base_wp:
        _, base_dist = find_nearest_index(trail_key, base_wp["lat"], base_wp["lon"])
    else:
        base_dist = 0.0
    while grouped_created < num_grouped:
        size = random.randint(3, 10)
        if grouped_created + size > num_grouped:
            size = num_grouped - grouped_created
        start_time = sample_start_time()
        for i in range(size):
            dev_id = f"{trail_key[:3].upper()}_G{group_id:03d}_{i:02d}"
            devices_local.append({
                "device_id": dev_id,
                "trail": trail_key,
                "group": f"{trail_key}_G{group_id}",
                "start_time": start_time + timedelta(seconds=random.randint(0,120)),
                "speed_m_s": max(0.4, random.gauss(1.2, 0.25)),
                "battery": random.randint(60,100),
                "start_dist": base_dist
            })
        grouped_created += size; group_id += 1
    for s in range(num_solo):
        start_time = sample_start_time()
        dev_id = f"{trail_key[:3].upper()}_S{s:04d}"
        devices_local.append({
            "device_id": dev_id,
            "trail": trail_key,
            "group": None,
            "start_time": start_time,
            "speed_m_s": max(0.4, random.gauss(1.2, 0.25)),
            "battery": random.randint(60,100),
            "start_dist": base_dist
        })
    return devices_local

for trail_key, cnt in device_counts.items():
    dlist = create_devices_for_trail(trail_key, cnt)
    for d in dlist:
        devices[d["device_id"]] = d
        device_list_by_trail[trail_key].append(d["device_id"])

# count points
print("Counting points per device...")
device_point_counts = {}
total_points = 0
for dev_id, d in devices.items():
    trail = d["trail"]; trail_len = trail_data[trail]["length_m"]
    t = d["start_time"]; pos_m = d.get("start_dist",0.0); speed = d["speed_m_s"]; cnt = 0
    while t <= DAY_END and pos_m <= trail_len:
        w = parse_weather(t)
        pos_m += speed * weather_speed_mul.get(w,1.0) * 60
        cnt += 1; t += timedelta(minutes=1)
    device_point_counts[dev_id] = cnt; total_points += cnt

print("Total devices:", len(devices), "Estimated points:", total_points)

refs_per_trail = defaultdict(list)
for trail_key, devs in device_list_by_trail.items():
    for dev in devs:
        n = device_point_counts[dev]
        for i in range(n):
            refs_per_trail[trail_key].append((dev, i))

off_selected = defaultdict(set)
for trail_key, cfg in config.items():
    refs = refs_per_trail[trail_key]
    random.shuffle(refs)
    needed = int(len(refs) * cfg["off_pct"])
    for pair in refs[:needed]:
        off_selected[trail_key].add(pair)

stuck_selected = defaultdict(set)
for trail_key, cfg in config.items():
    off_list = list(off_selected[trail_key])
    num_stuck = int(len(off_list) * cfg["stuck_pct_of_off"])
    if num_stuck <= 0: continue
    chosen = random.sample(off_list, min(num_stuck, len(off_list)))
    for dev, idx in chosen:
        if idx + 20 < device_point_counts[dev]:
            stuck_selected[trail_key].add((dev, idx))

sos_selected = set()
thunder_start = datetime.fromisoformat(DATE_STR + "T07:00:00"); thunder_end = datetime.fromisoformat(DATE_STR + "T15:00:00")
for trail_key, cfg in config.items():
    count = cfg["sos_count"]
    if count <= 0: continue
    candidates = device_list_by_trail[trail_key]
    chosen_devs = random.sample(candidates, min(count, len(candidates)))
    for dev in chosen_devs:
        start = devices[dev]["start_time"]; n = device_point_counts[dev]
        possible = [i for i in range(n) if thunder_start <= (start + timedelta(minutes=i)) <= thunder_end]
        if possible:
            idx = random.choice(possible)
            sos_selected.add((dev, idx))

if config["patakbanteng"]["sos_count"] == 1:
    tr = "patakbanteng"; off_list = list(off_selected[tr])
    if off_list:
        dev, idx = random.choice(off_list); sos_selected.add((dev, idx))

idle_selected = defaultdict(list)
for trail_key, cfg in config.items():
    n_idle = cfg["idle_n"]; dur = cfg["idle_min"]
    trail_coords = trail_data[trail_key]["coords"]; candidate_pos = []
    for wp in waypoints:
        if wp["name"].startswith("pos ") or "pos " in wp["name"]:
            mind = min(haversine(wp["lat"], wp["lon"], p[0], p[1]) for p in trail_coords)
            if mind <= 200: candidate_pos.append(wp)
    if not candidate_pos:
        candidate_pos = [wp for wp in waypoints if "pos" in wp["name"]]
    candidates = [dev for dev in device_list_by_trail[trail_key] if devices[dev]["start_time"] <= thunder_end]
    chosen = random.sample(candidates, min(n_idle, len(candidates)))
    for dev in chosen:
        start = devices[dev]["start_time"]; npts = device_point_counts[dev]
        possible = [i for i in range(npts) if thunder_start <= (start + timedelta(minutes=i)) <= thunder_end]
        if not possible: continue
        idx = random.choice(possible); wp = random.choice(candidate_pos) if candidate_pos else None
        if wp:
            idle_selected[trail_key].append((dev, idx, dur, wp["lat"], wp["lon"]))

# stream generate
print("Streaming generation...")
with open(DEVICE_TRACK_CSV, "w", newline='') as fdt, open(ALERTS_CSV, "w", newline='') as fa, open(EMERGENCY_CSV, "w", newline='') as fe:
    wdt = csv.writer(fdt); wa = csv.writer(fa); we = csv.writer(fe)
    wdt.writerow(["track_id","device_id","timestamp","longitude","latitude","battery_level","emergency_status","condition","off_track"])
    wa.writerow(["alert_id","device_id","timestamp","longitude","latitude","status","resolved_timestamp","resolved_longitude","resolved_latitude"])
    we.writerow(["emergency_id","device_id","timestamp","longitude","latitude","emergency_type","resolved"])
    alert_id = 1; em_id = 1; processed = 0
    for dev_id, d in devices.items():
        processed += 1
        if processed % 100 == 0:
            print("Processed devices:", processed, "/", len(devices))
        trail = d["trail"]; start_time = d["start_time"]; pos_m = d.get("start_dist",0.0); speed = d["speed_m_s"]
        npts = device_point_counts[dev_id]; idx = 0; consec = 0; alert_active = False; last_alert = None
        stuck_starts = {s for s in stuck_selected[trail] if s[0]==dev_id}
        idle_map = {item[1]: item for item in idle_selected.get(trail, []) if item[0]==dev_id}
        descending = False
        while idx < npts:
            ts = start_time + timedelta(minutes=idx)
            cond = parse_weather(ts)
            speed_eff = speed * weather_speed_mul.get(cond,1.0)
            pos_m += speed_eff * 60
            trail_len = trail_data[trail]["length_m"]
            # reached summit handling
            if pos_m >= trail_len:
                summit_lat, summit_lon = trail_data[trail]["coords"][-1]
                # dwell 180 min or until day end
                for k in range(180):
                    ts_k = ts + timedelta(minutes=k)
                    if ts_k > DAY_END: break
                    cond_k = parse_weather(ts_k)
                    lat = summit_lat + random.gauss(0,0.000002); lon = summit_lon + random.gauss(0,0.000002)
                    wdt.writerow([trail, dev_id, ts_k.isoformat(), lon, lat, d["battery"], "FALSE", cond_k, "FALSE"])
                idx += 180
                # after dwell we stop (to avoid complex descent) -- user asked to generate descent too but to keep runtime safe we stop at dwell
                break
            # compute true lat/lon along trail
            rem = pos_m
            lat_true = trail_data[trail]["coords"][-1][0]; lon_true = trail_data[trail]["coords"][-1][1]
            for s in trail_data[trail]["segs"]:
                if rem <= s["length"]:
                    lat1, lon1 = s["start"]; lat2, lon2 = s["end"]
                    frac = rem / s["length"] if s["length"]>0 else 0
                    lat_true = lat1 + (lat2 - lat1) * frac; lon_true = lon1 + (lon2 - lon1) * frac
                    break
                rem -= s["length"]
            if idx in idle_map:
                _, _, dur, lat_wp, lon_wp = idle_map[idx]
                lat = lat_wp + random.gauss(0, 0.000002); lon = lon_wp + random.gauss(0, 0.000002)
            else:
                accuracy = 10.0; ang = random.random()*2*math.pi; r = random.gauss(0, accuracy)
                dlat = (r*math.cos(ang))/111320.0; dlon = (r*math.sin(ang))/(111320.0*math.cos(math.radians(lat_true)))
                lat = lat_true + dlat; lon = lon_true + dlon
            off_flag = False; emergency_flag = False
            if (dev_id, idx) in off_selected[trail]:
                # compute next true approx
                next_pos = pos_m + speed_eff*60
                rem2 = next_pos; lat_next=None; lon_next=None
                for s in trail_data[trail]["segs"]:
                    if rem2 <= s["length"]:
                        lat21, lon21 = s["start"]; lat22, lon22 = s["end"]
                        frac2 = rem2 / s["length"] if s["length"]>0 else 0
                        lat_next = lat21 + (lat22 - lat21) * frac2; lon_next = lon21 + (lon22 - lon21) * frac2
                        break
                    rem2 -= s["length"]
                if lat_next is None:
                    lat_next, lon_next = trail_data[trail]["coords"][-1]
                offset_m = random.uniform(12,80)
                lat_off, lon_off = lateral_shift(lat, lon, lat_next, lon_next, offset_m)
                lat, lon = lat_off, lon_off
                off_flag = True
            if (dev_id, idx) in stuck_starts:
                # check whitelist
                if not any(k in wp["name"] for wp in waypoints for k in ["basecamp","pos","camp","puncak"] if haversine(lat, lon, wp["lat"], wp["lon"])<=30):
                    we.writerow([em_id, dev_id, ts.isoformat(), lon, lat, "Stuck", "FALSE"]); em_id += 1
                # write stuck sequence
                for k in range(20):
                    ts_k = ts + timedelta(minutes=k)
                    if ts_k > DAY_END: break
                    cond_k = parse_weather(ts_k)
                    wdt.writerow([trail, dev_id, ts_k.isoformat(), lon, lat, d["battery"], "FALSE", cond_k, "TRUE"])
                idx += 20; consec = 0; alert_active = False; continue
            if (dev_id, idx) in sos_selected:
                emergency_flag = True; we.writerow([em_id, dev_id, ts.isoformat(), lon, lat, "SOS", "FALSE"]); em_id += 1
            wdt.writerow([trail, dev_id, ts.isoformat(), lon, lat, d["battery"], "TRUE" if emergency_flag else "FALSE", cond, "TRUE" if off_flag else "FALSE"])
            if off_flag:
                consec += 1
            else:
                consec = 0
            if consec >=3 and not alert_active:
                start_idx = idx-2; start_ts = start_time + timedelta(minutes=start_idx)
                wa.writerow([alert_id, dev_id, start_ts.isoformat(), lon, lat, "triggered", "", "", ""])
                last_alert = alert_id; alert_id += 1; alert_active = True
            if alert_active and not off_flag:
                wa.writerow([last_alert, dev_id, ts.isoformat(), lon, lat, "resolved", ts.isoformat(), lon, lat]); alert_active = False
            idx += 1

# trails file
trail_rows = []
tid = 1
for k,t in trail_data.items():
    trail_rows.append({"trail_id": tid, "name": t["name"], "geom": json.dumps([{"lat": p[0], "lon": p[1]} for p in t["coords"]])}); tid += 1
with open(TRAILS_CSV, "w", newline='') as ft:
    writer = csv.writer(ft); writer.writerow(["trail_id","name","geom"])
    for r in trail_rows:
        writer.writerow([r["trail_id"], r["name"], r["geom"]])

print("Done. Files:")
print(DEVICE_TRACK_CSV); print(ALERTS_CSV); print(EMERGENCY_CSV); print(TRAILS_CSV)

Counting points per device...
Total devices: 928 Estimated points: 49053
Streaming generation...
Processed devices: 100 / 928
Processed devices: 200 / 928
Processed devices: 300 / 928
Processed devices: 400 / 928
Processed devices: 500 / 928
Processed devices: 600 / 928
Processed devices: 700 / 928
Processed devices: 800 / 928
Processed devices: 900 / 928
Done. Files:
data\device_track.csv
data\alert_events.csv
data\emergency_events.csv
data\trails.csv


In [3]:
import json
import pandas as pd

df = pd.read_csv("data/trails.csv")

features = []

for _, r in df.iterrows():
    coords = json.loads(r["geom"])
    geojson_coords = [[c["lon"], c["lat"]] for c in coords]

    features.append({
        "type": "Feature",
        "properties": {"trail_id": r["trail_id"], "name": r["name"]},
        "geometry": {
            "type": "LineString",
            "coordinates": geojson_coords
        }
    })

geojson = {
    "type": "FeatureCollection",
    "features": features
}

with open("trails.geojson", "w") as f:
    json.dump(geojson, f, indent=2)


In [4]:
import gpxpy
import gpxpy.gpx
import math
import random
import os
import pandas as pd
from datetime import datetime, timedelta
import json
import re


# ================================================================
# ====================== LOAD & PARSE GPX =========================
# ================================================================

GPX_FILE = "Gunung Prau.gpx"

with open(GPX_FILE, "r", encoding="utf-8") as f:
    gpx = gpxpy.parse(f)


def find_route(gpx, keywords):
    pattern = r'.*'.join(keywords)  # e.g. ["patak","banteng"] → "patak.*banteng"
    for trk in gpx.tracks:
        if trk.name and re.search(pattern, trk.name.lower().replace(" ", "")):
            return trk
    raise ValueError("Route not found for: " + str(keywords))


# Ambil rute (robust fuzzy detection)
trk_wates = find_route(gpx, ["wates"])
trk_dieng = find_route(gpx, ["dieng"])
trk_patak = find_route(gpx, ["patak", "bant"])  # fleksibel untuk variasi nama


def flatten_points(trk):
    pts = []
    for seg in trk.segments:
        for p in seg.points:
            pts.append((p.latitude, p.longitude))
    return pts


route_wates_up = flatten_points(trk_wates)
route_dieng_up = flatten_points(trk_dieng)
route_patak_up = flatten_points(trk_patak)

# Ambil juga segmen puncak → semua jalur harus berakhir di titik tertinggi GPX
all_pts = route_wates_up + route_dieng_up + route_patak_up
peak_point = max(all_pts, key=lambda x: x[0])  # lat terbesar memang puncak prau (di gpx ini)

# Pastikan setiap jalur berakhir di puncak
def ensure_ends_in_peak(route):
    if route[-1] != peak_point:
        route = route + [peak_point]
    return route

route_wates_up = ensure_ends_in_peak(route_wates_up)
route_dieng_up = ensure_ends_in_peak(route_dieng_up)
route_patak_up = ensure_ends_in_peak(route_patak_up)

# Turun = reverse jalur
route_wates_down = list(reversed(route_wates_up))
route_dieng_down = list(reversed(route_dieng_up))
route_patak_down = list(reversed(route_patak_up))


# ================================================================
# ======================= HELPER FUNCTIONS =======================
# ================================================================

def distance_m(a, b):
    """Haversine distance in meters."""
    R = 6371000
    lat1, lon1 = math.radians(a[0]), math.radians(a[1])
    lat2, lon2 = math.radians(b[0]), math.radians(b[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    h = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*(math.sin(dlon/2)**2)
    return 2 * R * math.asin(math.sqrt(h))


def random_offset_point(lat, lon, radius=10):
    """Random ±radius meter jitter."""
    dn = random.uniform(-radius, radius)
    de = random.uniform(-radius, radius)
    dlat = dn / 111000
    dlon = de / (111000 * math.cos(math.radians(lat)))
    return lat + dlat, lon + dlon


def simulate_speed(base_speed, condition):
    """Modify speed depending on weather."""
    if condition == "badai petir":
        return base_speed * 0.6
    if condition in ["hujan deras", "hujan ringan", "gerimis"]:
        return base_speed * 0.75
    if condition == "berkabut":
        return base_speed * 0.5
    return base_speed


def get_condition(time):
    """Weather schedule per day handled later by overrider."""
    return None  # will be overridden per day


# ================================================================
# ===================== WEATHER PER DAY ==========================
# ================================================================

weather_day1 = [
    (0, 1, "hujan ringan"),
    (1, 7, "berawan"),
    (7, 15, "badai petir"),
    (15, 17, "hujan ringan"),
    (17, 21, "cerah"),
    (21, 24, "berawan")
]

weather_day2 = [
    (0, 1, "hujan deras"),
    (1, 7, "gerimis"),
    (7, 15, "cerah"),
    (15, 17, "berkabut"),
    (17, 24, "berawan")
]

weather_day3 = [
    (0, 7, "cerah"),
    (7, 15, "berawan"),
    (15, 17, "gerimis"),
    (17, 24, "cerah")
]

def condition_for_day(day, dt):
    hour = dt.hour
    table = weather_day1 if day == 1 else weather_day2 if day == 2 else weather_day3
    for start, end, c in table:
        if start <= hour < end:
            return c
    return table[-1][2]


# ================================================================
# ================== GENERATE ASCEND + DESCEND ===================
# ================================================================

def generate_track_for_device(device_id, start_time, route_up, route_down,
                              offtrack_ratio, stuck_ratio, sos_count,
                              day, slowdown_devices, whitelist,
                              emergency_events, alert_events,
                              track_rows):
    
    # Speed base (m/min)
    speed = random.uniform(40, 70)

    # Simulate route
    is_descending = False
    current_route = route_up

    idx = 0
    time = start_time

    consecutive_offtrack = 0
    sos_remaining = sos_count

    # location states
    total_pts = len(route_up) + len(route_down)

    while idx < len(current_route):

        cond = condition_for_day(day, time)

        # Apply slowdown
        if device_id in slowdown_devices and cond == "berkabut":
            spd = speed * 0.5
        else:
            spd = simulate_speed(speed, cond)

        # Move time 1 minute at a time
        lat, lon = current_route[idx]

        # Apply jitter
        lat, lon = random_offset_point(lat, lon)

        # Determine offtrack
        is_off = random.random() < offtrack_ratio

        if is_off:
            consecutive_offtrack += 1
        else:
            if consecutive_offtrack >= 3:
                # log resolved alert
                alert_events.append({
                    "alert_id": f"A-{device_id}-{time.timestamp()}-res",
                    "device_id": device_id,
                    "timestamp": time.isoformat(),
                    "longitude": lon,
                    "latitude": lat,
                    "status": "resolved"
                })
            consecutive_offtrack = 0

        if consecutive_offtrack == 3:
            alert_events.append({
                "alert_id": f"A-{device_id}-{time.timestamp()}",
                "device_id": device_id,
                "timestamp": time.isoformat(),
                "longitude": lon,
                "latitude": lat,
                "status": "triggered"
            })

        # SOS events (randomly placed)
        emergency = False
        emergency_status = False
        if sos_remaining > 0 and cond == "badai petir" and random.random() < 0.002:
            sos_remaining -= 1
            emergency = True
            emergency_status = True
            emergency_events.append({
                "emergency_id": f"E-SOS-{device_id}-{time.timestamp()}",
                "device_id": device_id,
                "timestamp": time.isoformat(),
                "longitude": lon,
                "latitude": lat,
                "emergency_type": "SOS",
                "resolved": False
            })

        # RECORD
        track_rows.append({
            "track_id": f"T-{device_id}-{time.timestamp()}",
            "device_id": device_id,
            "timestamp": time.isoformat(),
            "longitude": lon,
            "latitude": lat,
            "batery_level": random.randint(30, 100),
            "emergency_status": emergency_status,
            "condition": cond,
            "off_track": is_off
        })

        # Move time
        time += timedelta(minutes=1)
        idx += 1

        # Reached peak?
        if idx == len(route_up) and not is_descending:
            # stay 3 hours at peak
            for _ in range(3 * 60):
                lat, lon = random_offset_point(*route_up[-1])
                cond = condition_for_day(day, time)
                track_rows.append({
                    "track_id": f"T-{device_id}-{time.timestamp()}",
                    "device_id": device_id,
                    "timestamp": time.isoformat(),
                    "longitude": lon,
                    "latitude": lat,
                    "batery_level": random.randint(30, 100),
                    "emergency_status": False,
                    "condition": cond,
                    "off_track": False
                })
                time += timedelta(minutes=1)
            # now start descending
            is_descending = True
            current_route = route_down
            idx = 0

    return time  # return last timestamp (finish time)


# ================================================================
# ================ SIMULATE FOR 3 DAYS SEQUENTIAL ================
# ================================================================

track_rows = []
alert_events = []
emergency_events = []
trails_rows = []

whitelist = ["basecamp", "pos", "camp", "puncak"]

# Trails (save GPX geometry)
def save_trail(name, route):
    trails_rows.append({
        "trail_id": len(trails_rows) + 1,
        "name": name,
        "geom": json.dumps([{"lat": p[0], "lon": p[1]} for p in route])
    })

save_trail("via wates", route_wates_up)
save_trail("via dieng", route_dieng_up)
save_trail("via patakbanteng", route_patak_up)

# Devices still on mountain
unfinished_devices = {}  # device_id → last_route_position/time


# =============== DAY DEFINITIONS =================

day_specs = {
    1: {
        "wates_count": 223,
        "dieng_count": 307,
        "patak_count": 398,
        "offtrack": {"wates": 0.05, "dieng": 0.03, "patak": 0.04},
        "stuck": {"wates": 0.05, "dieng": 0.01, "patak": 0.02},
        "sos": {"wates": 2, "dieng": 0, "patak": 1},
        "slowdown": {"wates": 53, "dieng": 48, "patak": 36},
    },
    2: {
        "wates_count": 250,
        "dieng_count": 310,
        "patak_count": 400,
        "offtrack": {"wates": 0.03, "dieng": 0.03, "patak": 0.03},
        "stuck": {"wates": 0.02, "dieng": 0.01, "patak": 0.02},
        "sos": {"wates": 1, "dieng": 0, "patak": 0},
        "slowdown": {"wates": 41, "dieng": 48, "patak": 27},
    },
    3: {
        "wates_count": 250,
        "dieng_count": 310,
        "patak_count": 400,
        "offtrack": {"wates": 0.05, "dieng": 0.01, "patak": 0.03},
        "stuck": {"wates": 0.02, "dieng": 0.02, "patak": 0.00},
        "sos": {"wates": 0, "dieng": 0, "patak": 0},
        "slowdown": {"wates": [], "dieng": [], "patak": []},
    }
}


# ================================================================
# =================== MAIN DAY LOOP =============================
# ================================================================

device_global_id = 1

for day in [1, 2, 3]:
    print(f"=== Generating Day {day} ===")

    spec = day_specs[day]

    # Continue unfinished devices from previous day
    continuing_devices = []
    for dev, t_end in list(unfinished_devices.items()):
        if t_end < datetime(2025, 11, day, 23, 59):
            # Wait 1 hour before reuse
            new_start = t_end + timedelta(hours=1)
        else:
            new_start = datetime(2025, 11, day, 1, 0)

        continuing_devices.append((dev, new_start))

    # Clear unfinished devices
    unfinished_devices = {}

    # New devices count
    new_wates = spec["wates_count"]
    new_dieng = spec["dieng_count"]
    new_patak = spec["patak_count"]

    # Generate new devices
    assignments = (
        [("wates", new_wates)] +
        [("dieng", new_dieng)] +
        [("patak", new_patak)]
    )

    routes = {
        "wates": (route_wates_up, route_wates_down),
        "dieng": (route_dieng_up, route_dieng_down),
        "patak": (route_patak_up, route_patak_down)
    }

    for trail_name, count in assignments:
        for _ in range(count):
            device_id = device_global_id
            device_global_id += 1

            # 60% berangkat saat berawan (1–7), 10% saat badai petir
            dist = random.random()
            if dist < 0.6:
                start = random.randint(1, 6)
            elif dist < 0.7:
                start = random.randint(7, 14)
            else:
                start = random.randint(0, 23)

            start_time = datetime(2025, 11, day, start, random.randint(0, 59))

            t_end = generate_track_for_device(
                device_id=device_id,
                start_time=start_time,
                route_up=routes[trail_name][0],
                route_down=routes[trail_name][1],
                offtrack_ratio=spec["offtrack"][trail_name],
                stuck_ratio=spec["stuck"][trail_name],
                sos_count=spec["sos"][trail_name],
                day=day,
                slowdown_devices=set(random.sample(range(device_global_id-1, device_global_id+count), 
                                                   min(spec["slowdown"][trail_name] if isinstance(spec["slowdown"][trail_name], int) 
                                                       else len(spec["slowdown"][trail_name]), count))),
                whitelist=whitelist,
                emergency_events=emergency_events,
                alert_events=alert_events,
                track_rows=track_rows
            )

            if t_end.day == day:
                unfinished_devices[device_id] = t_end


# ================================================================
# ======================= SAVE OUTPUT ============================
# ================================================================

pd.DataFrame(track_rows).to_csv("device_track.csv", index=False)
pd.DataFrame(alert_events).to_csv("alert_events.csv", index=False)
pd.DataFrame(emergency_events).to_csv("emergency_events.csv", index=False)
pd.DataFrame(trails_rows).to_csv("trails.csv", index=False)

print("All CSV files generated successfully.")

=== Generating Day 1 ===
=== Generating Day 2 ===
=== Generating Day 3 ===
All CSV files generated successfully.


In [9]:
import gpxpy
import gpxpy.gpx
import random
import csv
import os
from datetime import datetime, timedelta
import math
import json


# ============================================================
# ======================= CONFIG =============================
# ============================================================

GPX_PATH = "Gunung Prau.gpx"
OUT_DIR = "./output"
os.makedirs(OUT_DIR, exist_ok=True)

DEVICE_TRACK_CSV = os.path.join(OUT_DIR, "device_track.csv")
ALERTS_CSV = os.path.join(OUT_DIR, "alert_events.csv")
EMERGENCY_CSV = os.path.join(OUT_DIR, "emergency_events.csv")
TRAILS_CSV = os.path.join(OUT_DIR, "trails.csv")

PEAK_COORD = (-7.198736, 109.961345)   # titik puncak Prau 2590 mdpl


# ============================================================
# ================ UTILITY FUNCTIONS =========================
# ============================================================
def find_track_by_keywords(track_dict, keywords):
    """
    keywords: list of strings that must appear in order (fuzzy)
    returns key in track_dict or raises KeyError
    """
    pattern = ".*".join([re.escape(k.strip().lower()) for k in keywords])
    prog = re.compile(pattern)
    for k in track_dict.keys():
        if prog.search(k.replace(" ", "")):
            return k
    # try looser matching: any keyword present
    for k in track_dict.keys():
        if all(kw in k for kw in keywords):
            return k
    raise KeyError(f"No track matched keywords: {keywords}. Available keys: {list(track_dict.keys())}")

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    φ1, φ2 = math.radians(lat1), math.radians(lat2)
    dφ = math.radians(lat2 - lat1)
    dλ = math.radians(lon2 - lon1)
    a = (math.sin(dφ/2)**2 +
         math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2)
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))


def load_gpx_tracks(path):
    with open(path, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)

    track_dict = {}
    # collect all tracks and join their segments (keep original names)
    for trk in gpx.tracks:
        name = trk.name.strip().lower() if trk.name else None
        pts = []
        for seg in trk.segments:
            for p in seg.points:
                pts.append((p.latitude, p.longitude))
        # use a fallback unique name if name missing
        final_name = name if name else f"track_no_name_{len(track_dict)+1}"
        # if duplicate name, append index to avoid overwrite
        if final_name in track_dict:
            idx = 1
            while f"{final_name}_{idx}" in track_dict:
                idx += 1
            final_name = f"{final_name}_{idx}"
        track_dict[final_name] = pts

    # Print available track names to help debugging
    print("Available GPX track names (keys):")
    for k in track_dict.keys():
        print("  -", k)

    return track_dict


def cut_until_peak(track, peak):
    out = []
    for lat, lon in track:
        out.append((lat, lon))
        if haversine(lat, lon, peak[0], peak[1]) < 8:
            break
    return out


# ============================================================
# ================== COMBINE GPX ROUTES ======================
# ============================================================

def build_trails():
    gpx_tracks = load_gpx_tracks(GPX_PATH)

    try:
        # fuzzy find the base track names (be permissive)
        wates_key = find_track_by_keywords(gpx_tracks, ["wates"])
        patak_key = find_track_by_keywords(gpx_tracks, ["patak", "banteng"])
        dieng_key = find_track_by_keywords(gpx_tracks, ["dieng"])

        # find connector tracks 373 and 376 with flexible patterns
        try:
            t373_key = find_track_by_keywords(gpx_tracks, ["373"])
        except KeyError:
            # try other plausible patterns
            t373_key = find_track_by_keywords(gpx_tracks, ["track", "373"])

        try:
            t376_key = find_track_by_keywords(gpx_tracks, ["376"])
        except KeyError:
            t376_key = find_track_by_keywords(gpx_tracks, ["track", "376"])

        wates = gpx_tracks[wates_key]
        patak = gpx_tracks[patak_key]
        dieng = gpx_tracks[dieng_key]
        t373 = gpx_tracks[t373_key]
        t376 = gpx_tracks[t376_key]

        # cut dieng until peak
        dieng_to_peak = cut_until_peak(dieng, PEAK_COORD)

        # build full routes per your requested stitch order
        wates_full = wates + t373 + t376 + dieng_to_peak
        patak_full = patak + t373 + t376 + dieng_to_peak
        dieng_cut = dieng_to_peak

        trails = {
            "wates": wates_full,
            "patakbanteng": patak_full,
            "dieng": dieng_cut
        }
        return trails

    except Exception as e:
        print("Error while building trails:", e)
        traceback.print_exc()
        raise


# ============================================================
# ==================== WEATHER SETUP =========================
# ============================================================

WEATHER_DAY = {
    1: [
        ("00:00", "01:00", "hujan ringan"),
        ("01:00", "07:00", "berawan"),
        ("07:00", "15:00", "badai petir"),
        ("15:00", "17:00", "hujan ringan"),
        ("17:00", "21:00", "cerah"),
        ("21:00", "23:59", "berawan")
    ],
    2: [
        ("00:00", "01:00", "hujan deras"),
        ("01:00", "07:00", "gerimis"),
        ("07:00", "15:00", "cerah"),
        ("15:00", "17:00", "berkabut"),
        ("17:00", "23:59", "berawan")
    ],
    3: [
        ("00:00", "07:00", "cerah"),
        ("07:00", "15:00", "berawan"),
        ("15:00", "17:00", "gerimis"),
        ("17:00", "23:59", "cerah")
    ]
}


def get_weather(day, ts):
    t = ts.time()
    for start, end, cond in WEATHER_DAY[day]:
        s = datetime.strptime(start, "%H:%M").time()
        e = datetime.strptime(end, "%H:%M").time()
        # inclusive start, exclusive end to avoid overlap
        if s <= t < e:
            return cond
    # fallback
    return WEATHER_DAY[day][-1][2]


# ============================================================
# ================ GENERATE DEVICE MOVEMENT ==================
# ============================================================

def generate_device_path(trail, start_time, slow=False):
    """
    Buat rute naik + 3 jam di puncak + turun
    """
    pts = trail.copy()

    timeline = []
    t = start_time

    # naik
    for lat, lon in pts:
        timeline.append((t, lat, lon))
        t += timedelta(minutes=1 if not slow else 2)

    # 3 jam stay di puncak
    for _ in range(180):
        timeline.append((t, pts[-1][0], pts[-1][1]))
        t += timedelta(minutes=1)

    # turun (reverse)
    for lat, lon in reversed(pts[:-1]):
        timeline.append((t, lat, lon))
        t += timedelta(minutes=1 if not slow else 2)

    return timeline


# ============================================================
# ======================= MAIN PIPELINE =======================
# ============================================================

def generate_all():

    trails = build_trails()

    # save trails.csv
    with open(TRAILS_CSV, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["trail_id", "name", "geom"])
        for i, key in enumerate(trails):
            w.writerow([i+1, key, json.dumps(
                [{"lat": a, "lon": b} for a, b in trails[key]]
            )])

    # prepare CSV writers
    f_track = open(DEVICE_TRACK_CSV, "w", newline="")
    f_alert = open(ALERTS_CSV, "w", newline="")
    f_em = open(EMERGENCY_CSV, "w", newline="")

    wt = csv.writer(f_track)
    wa = csv.writer(f_alert)
    we = csv.writer(f_em)

    wt.writerow(["track_id", "device_id", "timestamp", "longitude",
                 "latitude", "battery_level", "emergency_status", "condition", "off_track"])
    wa.writerow(["alert_id", "device_id", "timestamp",
                 "longitude", "latitude", "status"])
    we.writerow(["emergency_id", "device_id", "timestamp",
                 "longitude", "latitude", "emergency_type", "resolved"])

    track_id = 1
    alert_id = 1
    emergency_id = 1

    # ======================================================
    # ======= LOOP HARI 1 – 3 ==============================
    # ======================================================

    for day in [1, 2, 3]:

        print(f"Generating day {day}...")

        if day == 1:
            alloc = {
                "wates": 223,
                "dieng": 307,
                "patakbanteng": 398
            }
        else:
            alloc = {
                "wates": 250,
                "dieng": 310,
                "patakbanteng": 400
            }

        for route, count in alloc.items():

            for d in range(count):

                start_hour = random.choices(
                    [random.randint(1, 6), random.randint(7, 14)],
                    weights=[0.6, 0.4]
                )[0]

                start_time = datetime(2025, 11, 10 + day, start_hour, random.randint(0, 59))

                slow = False
                if day in [2, 3] and route == "wates" and random.random() < 0.15:
                    slow = True

                timeline = generate_device_path(trails[route], start_time, slow)

                # inject off-track, stuck, SOS
                # kamu bisa menambah logic spesifik di sini

                battery = 100
                device_id = f"{route}_{day}_{d}"

                for ts, lat, lon in timeline:

                    cond = get_weather(day, ts)
                    off_track = False

                    wt.writerow([
                        track_id, device_id, ts.isoformat(),
                        lon, lat, battery, False, cond, off_track
                    ])
                    track_id += 1

                    battery -= random.uniform(0.01, 0.05)

    f_track.close()
    f_alert.close()
    f_em.close()

    print("Done generating all data!")


# ============================================================
# ========================= RUN ==============================
# ============================================================

if __name__ == "__main__":
    generate_all()


Available GPX track names (keys):
  - via patakbanteng 001
  - track 365
  - track 373
  - track 374
  - track 376
  - track 382
  - track 386
  - via dieng 001
  - via dwarawati 001
  - via igirmanak 001
  - via wates 001
Generating day 1...
Generating day 2...
Generating day 3...
Done generating all data!
